In [ ]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib
import folium as fl

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA = DIR / 'data'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# List feed

path = DATA / 'cairns_gtfs.zip'
gk.list_feed(path)

In [ ]:
# Read feed and describe

feed = gk.read_feed(path, dist_units='m')
feed.describe()


In [ ]:
# Add distances to stop times

display(feed.stop_times)
feed = feed.append_dist_to_stop_times()
display(feed.stop_times)

In [ ]:
# Choose study dates

week = feed.get_first_week()
dates = [week[4], week[6]]  # First Friday and Sunday
dates

In [ ]:
# Compute feed time series

trip_stats = feed.compute_trip_stats()
trip_stats.head().T
fts = feed.compute_feed_time_series(trip_stats, dates, freq='6h')
fts

In [ ]:
gk.downsample(fts, freq='12h')


In [ ]:
# Compute feed stats for first week

feed_stats = feed.compute_feed_stats(trip_stats, week)
feed_stats

In [ ]:
# Compute route time series

rts = feed.compute_route_time_series(trip_stats, dates, freq='12h')
rts

In [ ]:
# Slice time series

inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['110-423', '111-423']

rts.loc[:, (inds, rids)]

In [ ]:
# Slice again by cross-section

rts.xs(rids[0], axis="columns", level=1)

In [ ]:
# Compute trip locations for every hour

rng = pd.date_range('1/1/2000', periods=24, freq='h')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(dates[0], times)
loc.head()

In [ ]:
# Build a route timetable

route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, dates).T

In [ ]:
# Compute screen line counts

trip_id = "CNS2014-CNS_MUL-Weekday-00-4166247"  # A non-simple looping trip
m = feed.map_trips([trip_id], show_stops=True, show_direction=True)
screen_line = gp.read_file(DATA / 'cairns_screen_line.geojson')

# Add screen line to map. Folium, why isn't this easier?!
# Remove existing layer control and fit bounds
keys_to_remove = [
    key for key in m._children.keys() 
    if key.startswith('layer_control_') or key.startswith('fit_bounds_')
]
for key in keys_to_remove:
    m._children.pop(key)

# Create and add the new feature group of screen line
fg = fl.FeatureGroup(name="Screen lines")
fl.GeoJson(
    screen_line,     
    style_function=lambda feature: {
        'color': 'red',
        'weight': 2,
    },
).add_to(fg)
fg.add_to(m)

# Add a new layer control and fit bounds
fl.LayerControl().add_to(m)
m.fit_bounds(fg.get_bounds())

display(m)

# Show screen line counts restricted to trip
slc = feed.compute_screen_line_counts(screen_line, dates=dates)
slc.loc[lambda x: x["trip_id"] == trip_id]


In [ ]:
# Map routes

rsns = feed.routes["route_short_name"].iloc[2:4]
feed.map_routes(route_short_names=rsns, show_stops=True)
